# Regridding Last Millennium simulations: sic/sit/tas

Katie Brennan  
August 2020

In [1]:
import sys,os,copy

import numpy as np
import pickle

In [2]:
sys.path.insert(1,'/home/disk/kalman2/mkb22/LMR_lite/')
import LMR_lite_utils as LMRlite
import LMR_utils as lmr
import LMR_config

Loading information from datasets.yml
Loading information from grid_def.yml


In [33]:
cfile = '/home/disk/kalman2/mkb22/LMR_lite/configs/config_ccsm4_sit_brennan2020.yml'

proxies = 'pseudo'

savedir = ('/home/disk/p/mkb22/nobackup/LMR_output/common_era_experiments/experiments/pseudo/')

print('loading configuration...')
cfg = LMRlite.load_config_simple(cfile)

loading configuration...
Checking configuration ... 
OK!


In [34]:
# LOAD PRIOR ENSEMBLE: 
print('loading prior')
X, Xb_one = LMRlite.load_prior(cfg)
Xbp = Xb_one - Xb_one.mean(axis=1,keepdims=True)

loading prior
Reading file:  /home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/tas_sfc_Amon_CCSM4_past1000_085001-185012.nc
(12012, 192, 288)
indlat= 0  indlon= 1
Anomalies provided as the prior: Removing the temporal mean (for every gridpoint)...
tas : Global(monthly): mean= 8.072375e-07  , std-dev= 1.8899411
Averaging over month sequence<<: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
tas : Global(time-averaged): mean= 4.4424884352419226e-08  , std-dev= 0.8317386411161235
Reading file:  /home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/sic_sfc_OImon_CCSM4_past1000_085001-185012.nc
(12012, 180, 360)
indlat= 0  indlon= 1
Full field provided as the prior
sic : Global(monthly): mean= 13.445376  , std-dev= 32.317898
Averaging over month sequence<<: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
sic : Global(time-averaged): mean= 13.445537533002385  , std-dev= 31.06984518565762
Reading file:  /home/disk/kalman3/rtardif/LMR/data/model/ccsm4_last_millenium/sit_noMV_OImon_CCSM4_

In [35]:
# Save lat/lon of prior for later reloading and regridding of truth data
prior_lat_full = X.prior_dict['tas_sfc_Amon']['lat']
prior_lon_full = X.prior_dict['tas_sfc_Amon']['lon']

prior_sic_lat_full = X.prior_dict['sic_sfc_OImon']['lat']
prior_sic_lon_full = X.prior_dict['sic_sfc_OImon']['lon']

prior_sit_lat_full = X.prior_dict['sit_noMV_OImon']['lat']
prior_sit_lon_full = X.prior_dict['sit_noMV_OImon']['lon']
# prior_sit_lat_full = X.prior_dict['sit_sfc_OImon']['lat']
# prior_sit_lon_full = X.prior_dict['sit_sfc_OImon']['lon']

In [36]:

# check if config is set to regrid the prior
if cfg.prior.regrid_method:
    print('regridding prior...')
    # this function over-writes X, even if return is given a different name
    [X_regrid,Xb_one_new] = LMRlite.prior_regrid(cfg,X,Xb_one,verbose=True)
else:
    X_regrid.trunc_state_info = X.full_state_info

regridding prior...
0 55295
(55296, 200)
(55296, 2)
(184896, 2)
tas_sfc_Amon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 192 288
=> Full array:      -11.247562408447266 8.361244201660156 -0.019776800259318432 0.8379584254775313
=> Truncated array: -11.101190125702887 8.303932885114513 -0.019449816660995874 0.833048701621003
55296 120095
(64800, 200)
(64800, 2)
(184896, 2)
sic_sfc_OImon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 180 360
=> Full array:      0.0 99.99801635742188 13.455709348935276 31.081066713453712
=> Truncated array: 0.0 99.72157837827302 13.467102466477478 30.53805028448229
120096 184895
(64800, 200)
(64800, 2)
(184896, 2)
sit_noMV_OImon  : 2D lat/lon variable, truncating this variable
nlat,nlon: 180 360
=> Full array:      0.0 67.77120208740234 0.4126973636495976 1.3464391916903842
=> Truncated array: 0.0 43.445540144058015 0.4143014260711595 1.2610664790047388


In [37]:
# Load full prior for tas and regrid
prior_tas_orig = X.prior_dict['tas_sfc_Amon']['value']
nyears = prior_tas_orig.shape[0]

prior_regrid_tas_prep = np.transpose(np.reshape(prior_tas_orig,
                                                (prior_tas_orig.shape[0],       
                                                 prior_tas_orig.shape[1]*prior_tas_orig.shape[2])))

# Regrid full tas prior for proxy selection and verification: 
[prior_tas_regrid,lat_tas_new,lon_tas_new] = lmr.regrid_esmpy(cfg.prior.esmpy_grid_def['nlat'],
                                                        cfg.prior.esmpy_grid_def['nlon'],
                                                        nyears,
                                                        prior_regrid_tas_prep,
                                                        prior_lat_full,
                                                        prior_lon_full,
                                                        prior_lat_full.shape[0],
                                                        prior_lat_full.shape[1],
                                                        method=cfg.prior.esmpy_interp_method)

prior_tas_regrid = np.reshape(np.transpose(prior_tas_regrid),(nyears,
                                                              cfg.prior.esmpy_grid_def['nlat'],
                                                              cfg.prior.esmpy_grid_def['nlon'],))

In [38]:
# Load full prior for tas and regrid
prior_sic_orig = X.prior_dict['sic_sfc_OImon']['value']
nyears = prior_sic_orig.shape[0]

prior_regrid_sic_prep = np.transpose(np.reshape(prior_sic_orig,
                                                (prior_sic_orig.shape[0],       
                                                 prior_sic_orig.shape[1]*prior_sic_orig.shape[2])))

# Regrid full tas prior for proxy selection and verification: 
[prior_sic_regrid,lat_sic_new,lon_sic_new] = lmr.regrid_esmpy(cfg.prior.esmpy_grid_def['nlat'],
                                                        cfg.prior.esmpy_grid_def['nlon'],
                                                        nyears,
                                                        prior_regrid_sic_prep,
                                                        prior_sic_lat_full,
                                                        prior_sic_lon_full,
                                                        prior_sic_lat_full.shape[0],
                                                        prior_sic_lon_full.shape[1],
                                                        method=cfg.prior.esmpy_interp_method)

prior_sic_regrid = np.reshape(np.transpose(prior_sic_regrid),(nyears,
                                                              cfg.prior.esmpy_grid_def['nlat'],
                                                              cfg.prior.esmpy_grid_def['nlon'],))

In [39]:
# Load full prior for tas and regrid
prior_sit_orig = X.prior_dict['sit_noMV_OImon']['value']
#prior_sit_orig = X.prior_dict['sit_sfc_OImon']['value']
nyears = prior_tas_orig.shape[0]

prior_regrid_sit_prep = np.transpose(np.reshape(prior_sit_orig,
                                                (prior_sit_orig.shape[0],       
                                                 prior_sit_orig.shape[1]*prior_sit_orig.shape[2])))

# Regrid full tas prior for proxy selection and verification: 
[prior_sit_regrid,lat_sit_new,lon_sit_new] = lmr.regrid_esmpy(cfg.prior.esmpy_grid_def['nlat'],
                                                        cfg.prior.esmpy_grid_def['nlon'],
                                                        nyears,
                                                        prior_regrid_sit_prep,
                                                        prior_sit_lat_full,
                                                        prior_sit_lon_full,
                                                        prior_sit_lat_full.shape[0],
                                                        prior_sit_lon_full.shape[1],
                                                        method=cfg.prior.esmpy_interp_method)

prior_sit_regrid = np.reshape(np.transpose(prior_sit_regrid),(nyears,
                                                              cfg.prior.esmpy_grid_def['nlat'],
                                                              cfg.prior.esmpy_grid_def['nlon'],))

In [40]:
ccsm4_dir = '/home/disk/chaos/mkb22/Documents/SeaIceData/CCSM4/CCSM4_last_millennium/'
ccsm4_file = 'ccsm4_sic_sit_tas_20CRv2_850_1850_full.npz'

np.savez(ccsm4_dir+ccsm4_file, sic_ccsm4=prior_sic_regrid, 
         sit_ccsm4=prior_sit_regrid, tas_ccsm4=prior_tas_regrid, 
         lat_ccsm4 = lat_sit_new, lon_ccsm4 = lon_sit_new)

# mpi_dir = '/home/disk/chaos/mkb22/Documents/SeaIceData/MPI/'
# mpi_file = 'mpi_sic_sit_tas_20CRv2_850_1850_full.npz'

# np.savez(mpi_dir+mpi_file, sic_mpi=prior_sic_regrid, 
#          sit_mpi=prior_sit_regrid, tas_mpi=prior_tas_regrid, 
#          lat_mpi = lat_sit_new, lon_mpi = lon_sit_new)